# Pruebas Lectura - Escritura DynamoDB

In [1]:
import sys
import os

# Ruta al proyecto (ajusta si estás en otra carpeta)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

Data source:
- https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/programming-with-python.html
- https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html

In [2]:
import boto3

## Session Creation

In [3]:
session = boto3.Session(profile_name='HousyProject')
dynamodb = session.client('dynamodb')

## Write

In [22]:
#generamos nuevos datos
import uuid
from datetime import datetime, timezone

In [31]:
user_id = str(uuid.uuid4())
chat_id = str(uuid.uuid4())
primary_key = f'USER#{user_id}#CONV#{chat_id}'
tstamp = datetime.now(timezone.utc).isoformat().replace('+00:00', 'Z')

In [ ]:
# escribimos en dynamodb, se usa estructura DyanamoDB. 
dynamodb.put_item(
    TableName = 'ChatMessages',
    Item={
        'PK': {'S': primary_key},
        'SK': {'S': 'TIMESTAMP#'+tstamp},
        'message': {'S': 'Random message'},
        'role': {'S': 'user'},
        'metadata': {'M': {
            'model': {'S': 'no_model'},
            'source': {'S': 'test'},
            'tokens': {'N': '40'},
            'user_agent': {'S': 'python_sdk'}
            }
        } 
    }
)

{'ResponseMetadata': {'RequestId': '8MD8O4N5KE3M4H52N5NMK4O42NVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 24 Jun 2025 16:58:39 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8MD8O4N5KE3M4H52N5NMK4O42NVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

OBVS: pendiente probar Deserializer y Serializer ( Conversion json a dynamo json y viceversa )

25-06: Se prueba Serializer satisfactoriamente

## Prueba Serializer

In [54]:
from boto3.dynamodb.types import TypeSerializer

In [55]:
serializer = TypeSerializer()

In [33]:
Item={
        'PK': primary_key,
        'SK': 'TIMESTAMP#'+tstamp,
        'message': 'Random message 2',
        'role': 'user',
        'metadata': {
            'model': 'no_model',
            'source': 'test',
            'tokens': '40',
            'user_agent': 'python_sdk'
            } 
    }

In [56]:
serialized_item = {k: serializer.serialize(v) for k, v in Item.items()}

In [57]:
serialized_item

{'PK': {'S': 'USER#f5865d6c-f972-43ea-8855-63d1aa7de3a6#CONV#295b06f1-a2d2-481e-a7de-96b9086a5935'},
 'SK': {'S': 'TIMESTAMP#2025-06-24T17:13:06.075335Z'},
 'message': {'S': 'Random message 2'},
 'role': {'S': 'user'},
 'metadata': {'M': {'model': {'S': 'no_model'},
   'source': {'S': 'test'},
   'tokens': {'S': '40'},
   'user_agent': {'S': 'python_sdk'}}}}

## Read

In [ ]:
# obtenemos todos los mensajes de un usuario
response = dynamodb.query(
    TableName = 'ChatMessages',
    KeyConditionExpression = 'PK = :pk_val',
    #FilterExpression='#name = :name_val',
    ExpressionAttributeValues = {
        ':pk_val' : {'S' : 'USER#7d4a81fc-32cd-498a-b3d0-2f35a123be97#CONV#1c5a0cf3-a995-438a-9c7c-bd1271230dd7'}
    }
    # , ExpressionAttributeNames={
    #     '#name': 'name',
    # }
)

In [50]:
response

{'Items': [{'metadata': {'M': {'model': {'S': 'no_model'},
     'tokens': {'N': '40'},
     'source': {'S': 'test'},
     'user_agent': {'S': 'python_sdk'}}},
   'SK': {'S': 'TIMESTAMP#2025-06-24T16:41:45.551716Z'},
   'message': {'S': 'Random message'},
   'PK': {'S': 'USER#7d4a81fc-32cd-498a-b3d0-2f35a123be97#CONV#1c5a0cf3-a995-438a-9c7c-bd1271230dd7'},
   'role': {'S': 'user'}},
  {'metadata': {'M': {'model': {'S': 'no_model'},
     'tokens': {'N': '35'},
     'source': {'S': 'test'},
     'user_agent': {'S': 'python_sdk'}}},
   'SK': {'S': 'TIMESTAMP#2025-06-24T17:41:45.551716Z'},
   'message': {'S': 'Random response'},
   'PK': {'S': 'USER#7d4a81fc-32cd-498a-b3d0-2f35a123be97#CONV#1c5a0cf3-a995-438a-9c7c-bd1271230dd7'},
   'role': {'S': 'chatbot'}}],
 'Count': 2,
 'ScannedCount': 2,
 'ResponseMetadata': {'RequestId': 'VFE6RQ2I2ENG2BC0PQBE38BD8VVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 24 Jun 2025 17:42:55 GMT',
   

In [ ]:
# retrieving messages
message_log = []
for item in response['Items']:
    message_entry = {
        'role': item['role']['S'],
        'message': item['message']['S']
    }
    message_log.append(message_entry)

print(message_log)

[{'role': 'user', 'message': 'Random message'}, {'role': 'chatbot', 'message': 'Random response'}]


# Testing Funciones Personalizadas

In [4]:
from app.services import dynamodb_queries

## Obtencion de mensajes

In [8]:
response = dynamodb_queries.get_all_messages(dynamodb, 'USER#7d4a81fc-32cd-498a-b3d0-2f35a123be97#CONV#1c5a0cf3-a995-438a-9c7c-bd1271230dd7')

In [9]:
response

{'Items': [{'metadata': {'M': {'model': {'S': 'no_model'},
     'tokens': {'N': '40'},
     'source': {'S': 'test'},
     'user_agent': {'S': 'python_sdk'}}},
   'SK': {'S': 'TIMESTAMP#2025-06-24T16:41:45.551716Z'},
   'message': {'S': 'Random message'},
   'PK': {'S': 'USER#7d4a81fc-32cd-498a-b3d0-2f35a123be97#CONV#1c5a0cf3-a995-438a-9c7c-bd1271230dd7'},
   'role': {'S': 'user'}},
  {'metadata': {'M': {'model': {'S': 'no_model'},
     'tokens': {'N': '35'},
     'source': {'S': 'test'},
     'user_agent': {'S': 'python_sdk'}}},
   'SK': {'S': 'TIMESTAMP#2025-06-24T17:41:45.551716Z'},
   'message': {'S': 'Random response'},
   'PK': {'S': 'USER#7d4a81fc-32cd-498a-b3d0-2f35a123be97#CONV#1c5a0cf3-a995-438a-9c7c-bd1271230dd7'},
   'role': {'S': 'assistant'}}],
 'Count': 2,
 'ScannedCount': 2,
 'ResponseMetadata': {'RequestId': '64M0BPBTCFN6TBCVDMH1GFC8TBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 24 Jun 2025 23:56:22 GMT',
 

## Conversion a formato List[Dict] para contexto de Bedrock

In [ ]:
# Conver
messages = dynamodb_queries.response_to_conversation(response)
print(messages)

[{'role': 'user', 'content': [{'text': 'Random message'}]}, {'role': 'assistant', 'content': [{'text': 'Random response'}]}]


## Prueba de la clase ChatMessage
La clase ChatMessage se usa para Standarizar los tipos de mensaje 

In [75]:
Item

{'PK': 'USER#f5865d6c-f972-43ea-8855-63d1aa7de3a6#CONV#295b06f1-a2d2-481e-a7de-96b9086a5935',
 'SK': 'TIMESTAMP#2025-06-24T17:13:06.075335Z',
 'message': 'Random message 2',
 'role': 'user',
 'metadata': {'model': 'no_model',
  'source': 'test',
  'tokens': '40',
  'user_agent': 'python_sdk'}}

In [66]:
data = dynamodb_queries.ChatMessage(**Item)

In [67]:
data

ChatMessage(PK='USER#f5865d6c-f972-43ea-8855-63d1aa7de3a6#CONV#295b06f1-a2d2-481e-a7de-96b9086a5935', SK='TIMESTAMP#2025-06-24T17:13:06.075335Z', role='user', message='Random message 2', metadata={'model': 'no_model', 'source': 'test', 'tokens': '40', 'user_agent': 'python_sdk'})

In [77]:
# forcing data error
Item_2 = {'PK': 'USER#f5865d6c-f972-43ea-8855-63d1aa7de3a6#CONV#295b06f1-a2d2-481e-a7de-96b9086a5935',
            'SK': 'TIMESTAMP#2025-06-24T17:13:06.075335Z',
            'message': 'Random message 2',
            'role': 'user',
            'metadata': 'error_test'
            }


In [79]:
try:
    dynamodb_queries.ChatMessage(**Item_2)
except Exception as e:
    print(e)


1 validation error for ChatMessage
metadata
  Input should be a valid dictionary [type=dict_type, input_value='error_test', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/dict_type


In [74]:
serialized_data = dynamodb_queries.serialize_item(data)

In [80]:
serialized_data

{'PK': {'S': 'USER#f5865d6c-f972-43ea-8855-63d1aa7de3a6#CONV#295b06f1-a2d2-481e-a7de-96b9086a5935'},
 'SK': {'S': 'TIMESTAMP#2025-06-24T17:13:06.075335Z'},
 'role': {'S': 'user'},
 'message': {'S': 'Random message 2'},
 'metadata': {'M': {'model': {'S': 'no_model'},
   'source': {'S': 'test'},
   'tokens': {'S': '40'},
   'user_agent': {'S': 'python_sdk'}}}}

In [ ]:
# Prueba de escritura
dynamodb_queries.write_message(dynamodb=dynamodb, table_name='ChatMessages', serialized_item=serialized_data)

# DynamoDB metadata Test

In [5]:
from app.services.dynamodb_queries import get_latests_messages
from app.services.chatbot_engine import format_conversation, format_conversation_2

In [6]:
response = get_latests_messages(dynamodb, "USER#test_metadata_2#CONV#test_metadata_2", 10)

In [7]:
response

{'Items': [{'metadata': {'M': {'attribute1': {'S': 'att1'},
     'attribute2': {'S': 'att2'}}},
   'SK': {'S': 'TIMESTAMP#2025-07-07T18:51:52.757243Z'},
   'message': {'S': '¡Hola! Para ayudarte a encontrar la mejor opción inmobiliaria, considera tus necesidades: presupuesto, tamaño, ubicación y características deseadas. Investiga áreas con buena calidad de vida y transporte. Usa plataformas inmobiliarias para ver listados y compara precios. Si tienes dudas, no dudes en preguntar. ¡Estoy aquí para ayudarte a encontrar tu hogar ideal!'},
   'PK': {'S': 'USER#test_metadata_2#CONV#test_metadata_2'},
   'role': {'S': 'assistant'}},
  {'metadata': {'M': {'attribute1': {'S': 'att1'},
     'attribute2': {'S': 'att2'}}},
   'SK': {'S': 'TIMESTAMP#2025-07-07T18:51:51.506991Z'},
   'message': {'S': 'This is a test.'},
   'PK': {'S': 'USER#test_metadata_2#CONV#test_metadata_2'},
   'role': {'S': 'user'}}],
 'Count': 2,
 'ScannedCount': 2,
 'ResponseMetadata': {'RequestId': '622LPT3KVQ3823VP40L6LP

In [8]:
from app.models.ChatMessage import ChatHistoryResponse, ChatHistoryElement

In [10]:
formatted_conversation = format_conversation(response, verbose=False)
formatted_conversation

[{'role': 'user', 'message': 'This is a test.'},
 {'role': 'assistant',
  'message': '¡Hola! Para ayudarte a encontrar la mejor opción inmobiliaria, considera tus necesidades: presupuesto, tamaño, ubicación y características deseadas. Investiga áreas con buena calidad de vida y transporte. Usa plataformas inmobiliarias para ver listados y compara precios. Si tienes dudas, no dudes en preguntar. ¡Estoy aquí para ayudarte a encontrar tu hogar ideal!'}]

In [11]:
ChatHistoryResponse(history=formatted_conversation)

ChatHistoryResponse(history=[ChatHistoryElement(role='user', message='This is a test.', SK=None, metadata=None), ChatHistoryElement(role='assistant', message='¡Hola! Para ayudarte a encontrar la mejor opción inmobiliaria, considera tus necesidades: presupuesto, tamaño, ubicación y características deseadas. Investiga áreas con buena calidad de vida y transporte. Usa plataformas inmobiliarias para ver listados y compara precios. Si tienes dudas, no dudes en preguntar. ¡Estoy aquí para ayudarte a encontrar tu hogar ideal!', SK=None, metadata=None)])

In [11]:
from boto3.dynamodb.types import TypeDeserializer

In [12]:
deserializer = TypeDeserializer()
item= response['Items'][0]
deserializedItem = {
        k: deserializer.deserialize(v) 
        for k, v in item.items()
    }

In [14]:
deserializedItem.pop('SK')

'TIMESTAMP#2025-07-07T18:51:52.757243Z'

In [15]:
deserializedItem

{'metadata': {'attribute1': 'att1', 'attribute2': 'att2'},
 'message': '¡Hola! Para ayudarte a encontrar la mejor opción inmobiliaria, considera tus necesidades: presupuesto, tamaño, ubicación y características deseadas. Investiga áreas con buena calidad de vida y transporte. Usa plataformas inmobiliarias para ver listados y compara precios. Si tienes dudas, no dudes en preguntar. ¡Estoy aquí para ayudarte a encontrar tu hogar ideal!',
 'PK': 'USER#test_metadata_2#CONV#test_metadata_2',
 'role': 'assistant'}

In [12]:
history = format_conversation_2(response)
history[0]

ChatHistoryElement(role='user', message='This is a test.', SK=None, metadata=None)

In [15]:
history

[ChatHistoryElement(role='user', message='This is a test.', SK=None, metadata=None),
 ChatHistoryElement(role='assistant', message='¡Hola! Para ayudarte a encontrar la mejor opción inmobiliaria, considera tus necesidades: presupuesto, tamaño, ubicación y características deseadas. Investiga áreas con buena calidad de vida y transporte. Usa plataformas inmobiliarias para ver listados y compara precios. Si tienes dudas, no dudes en preguntar. ¡Estoy aquí para ayudarte a encontrar tu hogar ideal!', SK=None, metadata=None)]

In [16]:
ChatHistoryResponse(history=history)

ChatHistoryResponse(history=[ChatHistoryElement(role='user', message='This is a test.', SK=None, metadata=None), ChatHistoryElement(role='assistant', message='¡Hola! Para ayudarte a encontrar la mejor opción inmobiliaria, considera tus necesidades: presupuesto, tamaño, ubicación y características deseadas. Investiga áreas con buena calidad de vida y transporte. Usa plataformas inmobiliarias para ver listados y compara precios. Si tienes dudas, no dudes en preguntar. ¡Estoy aquí para ayudarte a encontrar tu hogar ideal!', SK=None, metadata=None)])